In [27]:
import os
import time
from functools import reduce
from collections import Counter
from pprint import pprint
from numba import jit
from time import sleep
from concurrent.futures import ProcessPoolExecutor

In [28]:
# folder_name = input("Fill in the folder name in current directory")
path = os.path.abspath('')
print(path)
os.chdir(path)

C:\Users\sopro\Desktop\паралельні-та-розподілені-обчислення\documents


In [29]:
def read_file(file_path):
    with open(file_path, 'r') as f:
        return f.read()
    
# @jit(nogil=True)
def count_unique_words(file):
    str = read_file(f"{path}\{file}")
    sleep(4)
    return Counter(str.split('\n'))

def merge_counters(counter1, counter2):
    return counter1 + counter2
    

In [46]:
start_time = time.time()
filelist = os.listdir()

if __name__ == '__main__':
    with ProcessPoolExecutor() as executor:
        file_word_counters = [val for val in executor.map(count_unique_words, filelist)]
    print('file_word_counters', file_word_counters)
    # total_word_counter = reduce(merge_counters, file_word_counters)

    print("Time elapsed: %s sec" % (time.time() - start_time))

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.